In [7]:
import micropip
await micropip.install("https://christian.amsuess.com/aiocoap-0.4.5.post0-py3-none-any.whl")
await micropip.install("https://christian.amsuess.com/LinkHeader-0.4.2-py3-none-any.whl")
import aiocoap
ctx = await aiocoap.Context.create_client_context()

In [8]:
from aiocoap import resource

class WhoAmI(resource.Resource):
    async def render_get(self, request):
        text = ["Used protocol: %s." % request.remote.scheme]

        text.append("Request came from %s." % request.remote.hostinfo)
        text.append("The server address used %s." % request.remote.hostinfo_local)

        claims = list(request.remote.authenticated_claims)
        if claims:
            text.append("Authenticated claims of the client: %s." % ", ".join(repr(c) for c in claims))
        else:
            text.append("No claims authenticated.")

        return aiocoap.Message(content_format=0,
                payload="\n".join(text).encode('utf8'))

root = resource.Site()
root.add_resource(['.well-known', 'core'],
    resource.WKCResource(root.get_resources_as_linkheader))
root.add_resource(['whoami'], WhoAmI())
ctx.serversite = root

In [9]:
req = aiocoap.Message(
    code=aiocoap.POST,
    uri="coaps+ws://rd.coap.amsuess.com/.well-known/rd?proxy=on&ep=jupyter")
await ctx.request(req).response

<aiocoap.Message at 0x1b393b8: no mtype, 2.04 Changed (no MID, token 095c) remote <WSRemote at 0x206cd90, hostinfo rd.coap.amsuess.com, local [::]:1234>>